# Making noise catalog




###  We pick a station (J8) from which to create the catalog of 30-second noise records, applying an STA/LTA filter to find and remove records with prominant icequakes. 

### We then sample records from other stations (J4, J5) based on the J8 catalog. The records are taken approximately every 10 minutes from June 14 to July 22nd, inclusive. 



_Created on Sat Jun 15 14:46:35 2019 ; Last updated June 25, 2022_



3. Get 60 sec clip every 10 min

    * winLen = 60 #sec, length of data window
    * increment = 5 #s
    * dT = 10 minutes
    
    
4. Use obspy STA/LTA to check for unambiguous icequakes
    * thrup = 10  #upper thershold sta/lta
    * cft = classic_sta_lta(wff.data, 50, 500) # in samples, 1000 Hz
 
                        


In [27]:





#%%


import obspy.signal 
from obspy import read
from obspy import UTCDateTime
import obspy as obs
import pandas as pd
import numpy as np
import os
import sys
sys.path.insert(0, '../01_DataPrep')
import random 
from matplotlib import pyplot as plt

import obspy.signal.filter
#import peak_detector_fcn as pkdetect
#import savitzky_golay as savitzky_golay
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import plot_trigger

# import insertName

#%%

In [28]:
!ls /Users/theresasawi/../../Volumes/Lyman/gorner_continuous/contRec_DOY_J8/

contRec_G7J8_doy165.sac contRec_G7J8_doy178.sac contRec_G7J8_doy191.sac
contRec_G7J8_doy166.sac contRec_G7J8_doy179.sac contRec_G7J8_doy192.sac
contRec_G7J8_doy167.sac contRec_G7J8_doy180.sac contRec_G7J8_doy193.sac
contRec_G7J8_doy168.sac contRec_G7J8_doy181.sac contRec_G7J8_doy194.sac
contRec_G7J8_doy169.sac contRec_G7J8_doy182.sac contRec_G7J8_doy195.sac
contRec_G7J8_doy170.sac contRec_G7J8_doy183.sac contRec_G7J8_doy196.sac
contRec_G7J8_doy171.sac contRec_G7J8_doy184.sac contRec_G7J8_doy197.sac
contRec_G7J8_doy172.sac contRec_G7J8_doy185.sac contRec_G7J8_doy198.sac
contRec_G7J8_doy173.sac contRec_G7J8_doy186.sac contRec_G7J8_doy199.sac
contRec_G7J8_doy174.sac contRec_G7J8_doy187.sac contRec_G7J8_doy200.sac
contRec_G7J8_doy175.sac contRec_G7J8_doy188.sac contRec_G7J8_doy201.sac
contRec_G7J8_doy176.sac contRec_G7J8_doy189.sac contRec_G7J8_doy202.sac
contRec_G7J8_doy177.sac contRec_G7J8_doy190.sac contRec_G7J8_doy203.sac


In [29]:
### Steps 3 and 4

fs = 1000 #sampling rate
#for sta in [1]:#,2,3,4,5,7,8]:
plot=0 #plot waveforms? 
fill_value = 1000000000.0 # we defined this as a missing value in the previous script 1A


#stations 
sta = Station = 'J8'

###paths IN
pathContIn = "/Users/theresasawi/../../Volumes/Lyman/gorner_continuous/contRec_DOY_J8/"#contRec_DOY/"#contRec_G7J4_doy167.sac"
# pathContIn4 = "/Users/theresasawi/../../Volumes/Lyman/gorner_continuous/contRec_DOY_J4/"#contRec_DOY/"#contRec_G7J4_doy167.sac"
# pathContIn5 = "/Users/theresasawi/../../Volumes/Lyman/gorner_continuous/contRec_DOY_J5/"#contRec_DOY/"#contRec_G7J4_doy167.sac"

# pathContIn = "/Users/theresasawi/Documents/gorner_2007_all/contRec_DOY_J8/"

###paths OUT
homedir = "/Users/theresasawi/Documents/SpecUFEx_v1/BB_Gorner_Cont_Redo/"
pathWFOut  = homedir + "01_input/" + Station + "/waveforms/"

# pathWFOut4  = "/Users/theresasawi/Documents/11_Manuscripts/gorner_preprocessing/data/processed/waveforms_J4/"
# pathWFOut5  = "/Users/theresasawi/Documents/11_Manuscripts/gorner_preprocessing/data/processed/waveforms_J5/"

pathCat    = homedir + "01_input/" + Station + "/catalogs/"
#     pathFigOut = homeDir + '01_input/' + Station + '/figures/01_waveforms/'
#     pathRejOut = homeDir + '01_input/' + Station + '/figures/01_waveforms/01_rejects/'
#     pathOther = homeDir + '01_input/' + Station + '/figures/03_other/'    
#     pathCat1 = homeDir + '01_input/labeled_datasets/'
#     pathCat2 = homeDir + '01_input/' + Station + '/catalogs/'






In [30]:



winLen = 60#sec, length of data window

increment_s = 5 #go forward 5 seconds and check again if peak found

dT = 10*60# try to get a window every ten minutes 



In [38]:
plot=0


year = []
month = []
day = []
hour = []
minute = []
second = []
ddoy_list= []
event_ID = []  ## of form 070614005959



# doy = [165]#range(165,204) ##FULLL RANGE
doy = range(165,203) ##FULLL RANGE

i=-1    

for ddoy in doy:

    
    filename = 'contRec_G7'+str(sta)+ '_doy' + str(ddoy)+ '.sac'


    print('Reading DOY ' + str(ddoy))

    ev = read(pathContIn + filename)

    print(ev)


    t0 = UTCDateTime(year=2007, julday=ddoy)

    t0_orig = t0 #save original start time        


    i +=1

    
    while t0.day == t0_orig.day:
#         print(i, t0)

        #increment start times 

        t1 = t0 + winLen
        
################### trim waveform ############################z
        ev_copy1 = ev.copy()
        try:
            wff = ev_copy1.trim(starttime=t0,endtime=t1)[0]                
#         print(wff)        
#         del ev
################################################################        
        
    
################### check for peaks ############################
    
            thrup = 10  #upper thershold sta/lta



            cft = classic_sta_lta(wff.data, 30, 500) #sta/lta; 


            if plot:
                if np.mod(i,10)==0:
                    plt.figure()
                    plt.title(t0)

                    plt.plot(wff.data)            
                    plot_trigger(wff, cft, thrup, 0.5)



                #if peak detected, or if all zeros or if any of data are the fill value, increment time and repeat:
            if len(cft[cft>thrup])>0 or np.any(wff.data == fill_value)==True or np.all(wff.data == 0)==True:# and np.max(wff.data) >= 5*np.mean(np.abs(wff.data)):

                isPeak=1
    #             print('peak detected')
                t0 = t1
                t1 += increment_s
                pass
            

################################################################                

            else :
                isPeak=0
                noiseFound=1  




                time = wff.stats.starttime
            #    try:

                y = str(time)[3:4]
                mo = str(time)[5:7]
                d = str(time)[8:10]
                h = str(time)[11:13]
                mi = str(time)[14:16]
                s = str(time)[17:19]

                year.append(str(y))    
                month.append(int(mo))
                day.append(int(d))
                hour.append(int(h))
                minute.append(int(mi))
                second.append(int(s))
                ddoy_list.append(ddoy)

                idd = str(y) + str(mo) + str(d) + str(h) + str(mi) + str(s)                          
                event_ID.append(idd)

                ##save wf to file, and every 100 figures
    #                try:
    #             print('writing', i)
                wff.write(pathWFOut + idd + '.sac', format='SAC') 


                t0 = t1+dT
                t1 = t0 + winLen
        except:
#             print('unable to load wff')
            t0 = t1
            t1 += increment_s            
            pass
            

        

Reading DOY 165
1 Trace(s) in Stream:
4D.G7J8..SHZ | 2007-06-14T00:00:00.000000Z - 2007-06-15T00:00:00.004273Z | 1000.0 Hz, 86400000 samples
Reading DOY 166
1 Trace(s) in Stream:
4D.G7J8..SHZ | 2007-06-15T00:00:00.000000Z - 2007-06-16T00:00:00.004273Z | 1000.0 Hz, 86400000 samples
Reading DOY 167
1 Trace(s) in Stream:
4D.G7J8..SHZ | 2007-06-16T00:00:00.000000Z - 2007-06-17T00:00:00.004273Z | 1000.0 Hz, 86400000 samples
Reading DOY 168
1 Trace(s) in Stream:
4D.G7J8..SHZ | 2007-06-17T00:00:00.000000Z - 2007-06-18T00:00:00.004273Z | 1000.0 Hz, 86400000 samples
Reading DOY 169
1 Trace(s) in Stream:
4D.G7J8..SHZ | 2007-06-18T00:00:00.000000Z - 2007-06-19T00:00:00.004273Z | 1000.0 Hz, 86400000 samples
Reading DOY 170
1 Trace(s) in Stream:
4D.G7J8..SHZ | 2007-06-19T00:00:00.000000Z - 2007-06-20T00:00:00.004273Z | 1000.0 Hz, 86400000 samples
Reading DOY 171
1 Trace(s) in Stream:
4D.G7J8..SHZ | 2007-06-20T00:00:00.000000Z - 2007-06-21T00:00:00.004273Z | 1000.0 Hz, 86400000 samples
Reading DOY 1

In [39]:
def datenum2datetime(datenum):
    
    """
    Convert Gorner 2007 datenum to pandas timestamp
    """
    
    ev = str(datenum)
    y = '2007'
    m  = ev[1:3]
    d  = ev[3:5]
    h  = ev[5:7]
    mm = ev[7:9]
    s  = ev[9:11]
    
    datestr = f"{y}-{m}-{d} {h}:{mm}:{s}"
    
    date = pd.Timestamp(datestr)
    
    return date

In [40]:
cat_labeled = pd.DataFrame({'year'  : year, 
                            'month'  : month,
                            'day'    : day, 
                            'hour'   : hour,
                            'mn' : minute,
                            'sec' : second,
                            'event_ID'   : event_ID,
                            'doy'  : ddoy_list
                           })




In [46]:
## TEMP -- we will actually make this cat_labeled script here!! not load it .... 
cat_labeled['datetime_index'] = [datenum2datetime(d) for d in cat_labeled['event_ID']]
cat_labeled['datetime'] = [datenum2datetime(d) for d in cat_labeled['event_ID']]
cat_labeled.set_index('datetime_index',inplace=True)
cat_labeled['event_ID'] = [int(ev) for ev in cat_labeled['event_ID']]
cat_labeled.sort_index(inplace=True)
cat_labeled.to_csv(pathCat + f"catalog_G7J8Z.csv")

In [45]:
pathCat

'/Users/theresasawi/Documents/SpecUFEx_v1/BB_Gorner_Cont_Redo/01_input/J8/catalogs/'

# Get same waveforms from station 5

In [43]:
import glob

In [44]:
station5_files = glob.glob(pathContIn5 + '*.sac')

sta8_waveforms_files = glob.glob(pathWFOut + "*.sac")

NameError: name 'pathContIn5' is not defined

In [41]:
!ls pathContIn5

ls: pathContIn5: No such file or directory


# Get same waveforms from J5

In [43]:


for doy in list(np.unique(cat_labeled.doy)):

    ev = read(f"{pathContIn5}contRec_G7J5_doy{doy}.sac")    

    cat_doy = cat_labeled[cat_labeled.doy == doy]

    for i, date in enumerate(cat_doy.datetime):

        ev_copy1 = ev.copy()
        
        evID = str(int(cat_doy.event_ID.iloc[i]))


        t0 = UTCDateTime(date)

        tev = t0+winLen
#         print(date, t0, tev)        

        wff = ev_copy1.trim(starttime=t0,endtime=tev)[0]  

        wff.write(pathWFOut5 + evID + '.sac', format='SAC')   


# Get same waveforms from J4

In [44]:


for doy in list(np.unique(cat_labeled.doy)):
    
    print(doy)

    ev = read(f"{pathContIn4}contRec_G7J4_doy{doy}.sac")    

    cat_doy = cat_labeled[cat_labeled.doy == doy]

    for i, date in enumerate(cat_doy.datetime):

        ev_copy1 = ev.copy()
        
        evID = str(int(cat_doy.event_ID.iloc[i]))


        t0 = UTCDateTime(date)

        tev = t0+winLen
#         print(date, t0, tev)        

        wff = ev_copy1.trim(starttime=t0,endtime=tev)[0]  

        wff.write(pathWFOut4 + evID + '.sac', format='SAC')   
        del ev_copy1

    del ev

165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202


In [45]:
cat_labeled

,year,month,day,hour,mn,sec,event_ID,doy,datetime
datetime_index,,,,,,,,,
2007-06-14 00:40:00,7,6,14,0,40,0,70614004000,165,2007-06-14 00:40:00
2007-06-14 01:07:00,7,6,14,1,7,0,70614010700,165,2007-06-14 01:07:00
2007-06-14 01:55:00,7,6,14,1,55,0,70614015500,165,2007-06-14 01:55:00
2007-06-14 03:27:59,7,6,14,3,27,59,70614032759,165,2007-06-14 03:27:59
2007-06-14 04:52:00,7,6,14,4,52,0,70614045200,165,2007-06-14 04:52:00
...,...,...,...,...,...,...,...,...,...
2007-07-21 22:42:59,7,7,21,22,42,59,70721224259,202,2007-07-21 22:42:59
2007-07-21 22:56:00,7,7,21,22,56,0,70721225600,202,2007-07-21 22:56:00
2007-07-21 23:36:00,7,7,21,23,36,0,70721233600,202,2007-07-21 23:36:00
